In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/qp/3cxfqxvd0dq7rn_xc3tvtcz00000gp/T/ipykernel_40161/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/zone/Desktop/Week 4/Day 2/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

data.info()

(1000, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1000 non-null   object
 1   label   1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


### Let's divide the training and test set into two partitions

In [4]:
from sklearn.model_selection import train_test_split

# Assign the 'text' column (SMS messages) to X as features and 'label' column (spam/ham classification) to y as target labels

X = data['text']
y = data['label']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=42)


## Data Preprocessing

In [5]:
import string
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to /Users/zone/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [6]:
from bs4 import BeautifulSoup, Comment  # Import  Beautiful Soup, library that helps parse and remove specific elements. Import Comment for detecting HTML comments
import re

# Sample HTML content
html_content = """
<html>
    <head>
        <style>body { font-size: 14px; }</style>
        <script type="text/javascript">alert('Hi!');</script>
    </head>
    <body>
        <!-- This is a comment -->
        <h1>Hello, World!</h1>
        <p>This is a paragraph.</p>
    </body>
</html>
"""

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Step 1: Remove inline JavaScript/CSS by removing <script> and <style> tags
for script_or_style in soup(["script", "style"]):
    script_or_style.decompose()  # Removes the tag and its contents

# Step 2: Remove HTML comments
comments = soup.find_all(string=lambda text: isinstance(text, Comment))
for comment in comments:
    comment.extract()  # Removes the comment

# Step 3: Remove all remaining HTML tags
cleaned_text = soup.get_text()


print(cleaned_text)










Hello, World!
This is a paragraph.





- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [7]:
import re

# Apply the cleaning function to every element in the X column (text data)
def clean_text(text):
    # Step 1: Remove all special characters (keeping only alphanumeric and space)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Step 2: Remove numbers
    text = re.sub(r'\d+', '', text)

    # Step 3: Remove all single characters (isolated single letters)
    text = re.sub(r'\b\w\b', '', text)

    # Step 4: Remove single characters from the start of the string
    text = re.sub(r'^\w\s', '', text)

    # Step 5: Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Step 6: Remove prefixed 'b'
    text = re.sub(r'^b', '', text)

    # Step 7: Convert to lowercase
    text = text.lower().strip()

    return text

# Apply the cleaning function to every element in X
X = X.apply(clean_text)

print(X)



0      dear sir strictly private business proposal am...
1                                                will do
2      noracheryl has emailed dozens of memos about h...
3      dear sirfmadamc know that this proposal might ...
4                                                    fyi
                             ...                        
995    so whats the latest it sounds contradictory an...
996    transfer of million pounds to youraccountmy na...
997    barb will call to explain are you back in the ...
998       yang on travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh just talked ...
Name: text, Length: 1000, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [8]:
from nltk.corpus import stopwords

# Download stopwords from NLTK
nltk.download('stopwords')

# Get the English stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords
def remove_stopwords(text):
    # Split the text into words and remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text


# Apply stopword removal function to every element in X
X = X.apply(remove_stopwords)


print(X)

0      dear sir strictly private business proposal mi...
1                                                       
2      noracheryl emailed dozens memos haiti weekend ...
3      dear sirfmadamc know proposal might surprise e...
4                                                    fyi
                             ...                        
995    whats latest sounds contradictory af decide sh...
996    transfer million pounds youraccountmy name mre...
997                       barb call explain back country
998          yang travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh talked shaun...
Name: text, Length: 1000, dtype: object


[nltk_data] Downloading package stopwords to /Users/zone/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [12]:
nltk.download('wordnet') 
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download the necessary NLTK data
nltk.download('punkt')      # Required for tokenization
nltk.download('wordnet')    # Required for lemmatization
nltk.download('omw-1.4')    # Additional wordnet resources (optional)

# Tokenize words, convert to lowercase and remove punctuation
from nltk.tokenize import word_tokenize

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to tokenize, remove punctuation, and lemmatize
def tokenizer_and_remove_punctuation(text):
      # Tokenize the text
  tokens = word_tokenize(text)
      # Remove non-alphabetic tokens and convert to lowercase
  tokens_cleaned = [word.lower() for word in tokens if word.isalpha()]
   # Lemmatize each token
  lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens_cleaned]

   # Join tokens back into a single string
  return ' '.join(lemmatized_tokens)  


X = X.apply(tokenizer_and_remove_punctuation)

print(X)

[nltk_data] Downloading package wordnet to /Users/zone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/zone/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/zone/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /Users/zone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/zone/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0      dear sir strictly private business proposal mi...
1                                                       
2      noracheryl emailed dozen memo haiti weekend pl...
3      dear sirfmadamc know proposal might surprise e...
4                                                    fyi
                             ...                        
995    whats latest sound contradictory af decide sha...
996    transfer million pound youraccountmy name mrej...
997                       barb call explain back country
998          yang travelnot free tonitemay work tomorrow
999    sbwhoeopsunday february pmhshaunh talked shaun...
Name: text, Length: 1000, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code